In [10]:
from scipy.optimize import linprog
import numpy as np
import matplotlib.pyplot as plt

# Параметры задачи
num_days = 5

# Коэффициенты целевых функций
c1 = [1 if i % 3 == 0 else -1 if i % 3 == 1 else 0 for i in range(3 * num_days)]  # f1: |x_A - x_B|
c2 = [0, 0, 1] * num_days  # f2: вечерние занятия

# Матрица ограничений
A_eq = [
    [1, 1, 0] * num_days  # x_A + x_B == 29
]

b_eq = [29]  # Общее число занятий

A_ub = [
    [1, 1, 0] * num_days,  # x_A + x_B <= 8 (по дням)
    [0, 0, 1] * num_days   # x_вечер <= 10
]
b_ub = [8] * num_days + [3]

# Границы переменных
bounds = [(0, None) for _ in range(3 * num_days)]

# Решение для f1
res_f1 = linprog(c1, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
print("Минимизация f1 (без учета f2):", res_f1.fun, res_f1.x)

# Уступки для f2
delta_f2 = np.linspace(0, 3, 5)  # Варианты уступок для f2
results = []

for delta in delta_f2:
    A_ub_with_delta = A_ub + [[0, 0, 1] * num_days]  # Ограничение для f2
    b_ub_with_delta = b_ub + [delta]

    res = linprog(c1, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub_with_delta, b_ub=b_ub_with_delta, bounds=bounds, method='highs')
    results.append((delta, res.fun, res.x))

# График
z1_values = [r[1] for r in results]
plt.plot(delta_f2, z1_values, label='f1 vs Delta f2')
plt.xlabel('Delta f2 (уступка)')
plt.ylabel('f1')
plt.legend()
plt.show()

ValueError: Invalid input for linprog: b_ub must be a 1-D array; b_ub must not have more than one non-singleton dimension and the number of rows in A_ub must equal the number of values in b_ub

In [11]:
# Веса для комбинированной функции
weights = np.linspace(0, 1, 10)
results_weights = []

for w1 in weights:
    w2 = 1 - w1
    c_combined = [w1 * c1[i] + w2 * c2[i] for i in range(len(c1))]

    res = linprog(c_combined, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
    results_weights.append((w1, res.fun, res.x))

# График
z_combined_values = [r[1] for r in results_weights]
plt.plot(weights, z_combined_values, label='Combined Z vs Weights')
plt.xlabel('w1 (вес f1)')
plt.ylabel('Combined Objective Z')
plt.legend()
plt.show()

ValueError: Invalid input for linprog: b_ub must be a 1-D array; b_ub must not have more than one non-singleton dimension and the number of rows in A_ub must equal the number of values in b_ub